In [1]:
from graph_of_thoughts.prompter import LanguageModelPrompter
from graph_of_thoughts.language_model import ChatGPT
lm = ChatGPT()
list = "[9, 6, 7, 7, 2, 0, 2, 2, 3, 5, 0, 9, 2, 2, 4, 4, 5, 2, 5, 1, 2, 8, 3, 8, 3, 9, 6, 0, 4, 2, 2, 3]"
main_task = f"Sort this list {list}"
prompter = LanguageModelPrompter(lm, main_task)
num_split = 4
print(f"Sort this list using sort algorithm")




Sort this list using sort algorithm


In [2]:
split_prompt = prompter.split_prompt({"state": f"START", "num_split": num_split, "current": "START"})
print(split_prompt)
split_result = lm.get_response_texts(lm.query(split_prompt))[0]
print(split_result)


<Instruction>NEXT TASK: Divide the list [9, 6, 7, 7, 2, 0, 2, 2, 3, 5, 0, 9, 2, 2, 4, 4, 5, 2, 5, 1, 2, 8, 3, 8, 3, 9, 6, 0, 4, 2, 2, 3] into 4 equal sublists.</Instruction> 
 INPUT: START 
 PROBLEM: Sort this list [9, 6, 7, 7, 2, 0, 2, 2, 3, 5, 0, 9, 2, 2, 4, 4, 5, 2, 5, 1, 2, 8, 3, 8, 3, 9, 6, 0, 4, 2, 2, 3] 
 OUTPUT:
<S>[0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2]</S>
<S>[3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6]</S>
<S>[7, 7, 8, 8, 9, 9, 9]</S>
<S>[2, 2, 2, 3, 5]</S>


In [3]:
split_result = split_result.replace("<S>","").replace("</S>", "").split("\n")


In [4]:
generate_result =[]
for v in split_result:
    generate_prompt = prompter.generate_prompt({"state": f"""Divide the given list into 4 equal sublists.""", "current":v})
    print(generate_prompt)
    result = lm.get_response_texts(lm.query(generate_prompt))[0]
    print(result)
    generate_result.append(result)

<Instruction>Sort the result of the previous task in ascending order. </Instruction> 
 INPUT: [0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2] 
 PROBLEM: Sort this list [9, 6, 7, 7, 2, 0, 2, 2, 3, 5, 0, 9, 2, 2, 4, 4, 5, 2, 5, 1, 2, 8, 3, 8, 3, 9, 6, 0, 4, 2, 2, 3] 
 OUTPUT:
<S>[0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2]</S>
<Instruction>Sort the result of the previous task in ascending order. </Instruction> 
 INPUT: [3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6] 
 PROBLEM: Sort this list [9, 6, 7, 7, 2, 0, 2, 2, 3, 5, 0, 9, 2, 2, 4, 4, 5, 2, 5, 1, 2, 8, 3, 8, 3, 9, 6, 0, 4, 2, 2, 3] 
 OUTPUT:
<S>[3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6]</S>
<Instruction>Sort the result of the previous task in ascending order. </Instruction> 
 INPUT: [7, 7, 8, 8, 9, 9, 9] 
 PROBLEM: Sort this list [9, 6, 7, 7, 2, 0, 2, 2, 3, 5, 0, 9, 2, 2, 4, 4, 5, 2, 5, 1, 2, 8, 3, 8, 3, 9, 6, 0, 4, 2, 2, 3] 
 OUTPUT:
<S>[7, 7, 8, 8, 9, 9, 9]</S>
<Instruction>Sort the sublist [2, 2, 2, 3, 5]. </Instruction> 
 INPUT: [2, 2, 2, 3, 5] 
 PROBLEM: Sort this list [9, 6

In [5]:
generate_result = "\n".join([f"""{e}: {v.replace("<S>","").replace("</S>", "")}""" for e, v in enumerate(generate_result)])

In [6]:
generate_prompt = """
1: Sort the result of the previous task in ascending order.
2: Sort the result of the previous task in ascending order.
3: Sort the result of the previous task in ascending order.
4: Sort the sublist [2, 2, 2, 3, 5]."""

In [7]:
print(generate_result)

0: [0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2]
1: [3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6]
2: [7, 7, 8, 8, 9, 9, 9]
3: [2, 2, 2, 3, 5]


In [8]:
aggregate_prompt = prompter.aggregate_prompt({"state": generate_prompt, "current":generate_result})
print(aggregate_prompt)
aggregate_result = lm.get_response_texts(lm.query(aggregate_prompt))[0]
print(aggregate_result)


<Instruction>Combine all the sorted sublists into a single sorted list. </Instruction> 
 INPUT: 0: [0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2]
1: [3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6]
2: [7, 7, 8, 8, 9, 9, 9]
3: [2, 2, 2, 3, 5] 
 PROBLEM: Sort this list [9, 6, 7, 7, 2, 0, 2, 2, 3, 5, 0, 9, 2, 2, 4, 4, 5, 2, 5, 1, 2, 8, 3, 8, 3, 9, 6, 0, 4, 2, 2, 3] 
 OUTPUT:
<S>[0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 9, 9, 9]</S>
